In [21]:
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

## Plot
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
import matplotlib as plt

# NLTK
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

# Other
import re
import string
import numpy as np
import pandas as pd
from sklearn.manifold import TSNE

In [22]:
#all the data
clickbait_data = pd.read_json("train.json", lines=True)
clickbait_data1 = pd.read_json("instances1.json", lines=True)
clickbait_data = clickbait_data.append(clickbait_data1)

In [23]:
clickbait_data

,id,postMedia,postText,postTimestamp,targetCaptions,targetDescription,targetKeywords,targetParagraphs,targetTitle
0,608310377143799808,[],[Apple's iOS 9 'App thinning' feature will giv...,Tue Jun 09 16:31:10 +0000 2015,['App thinning' will be supported on Apple's i...,'App thinning' will be supported on Apple's iO...,"Apple,gives,gigabytes,iOS,9,app,thinning,featu...",[Paying for a 64GB phone only to discover that...,Apple gives back gigabytes: iOS 9 'app thinnin...
1,609297109095972864,[media/609297109095972864.jpg],[RT @kenbrown12: Emerging market investors are...,Fri Jun 12 09:52:05 +0000 2015,"[Stocks Fall as Investors Watch Central Banks,...",Global investors have yanked $9.3 billion from...,"emerging market,emerging markets,em flows,em i...","[Emerging markets are out of favor., Global in...",Emerging Markets Suffer Largest Outflow in Sev...
2,609504474621612032,[],[U.S. Soccer should start answering tough ques...,Fri Jun 12 23:36:05 +0000 2015,[US to vote for Ali in FIFA election and not B...,A U.S. Senator's scathing letter questioned U....,,"[WINNIPEG, Manitoba – The bubble U.S. Soccer i...",U.S. Soccer should start answering tough quest...
3,609748367049105408,[],[How theme parks like Disney World left the mi...,Sat Jun 13 15:45:13 +0000 2015,"[Some 1,000 persons turned out in Albuquerque,...","America's top family vacation spots, like the ...","disney, disney world, disney ticket prices, di...",[When Walt Disney World opened in an Orlando s...,How theme parks like Disney World left the mid...
4,608688782821453824,[media/608688782821453825.jpg],[Could light bulbs hurt your health? One compa...,Wed Jun 10 17:34:49 +0000 2015,[Electric lights have made the world safer and...,One company will put a health notice on all th...,"health, Should there be warning labels on your...",[(CNN)The light bulb always makes the world's ...,Warning labels on your light bulbs
5,609551038983475200,[media/609551038983475201.png],[13 classic ’00s songs that were actually mean...,Sat Jun 13 02:41:07 +0000 2015,[],,,[One artist’s trash is another’s No. 1 single....,13 Classic ’00s Songs That Were Actually Meant...
6,609447408955719680,[media/609447408955719681.jpg],[Dez Bryant is reportedly considering skipping...,Fri Jun 12 19:49:19 +0000 2015,[],"Wide receiver Dez Bryant has signed a new,...","Football, NFL, NFC East, Dallas Cowboys, Dez B...","[Wide receiver Dez Bryant has signed a new, lo...","Dez Bryant Contract: Latest News, Rumors, Spec..."
7,609027430624288768,[],[Pregnant mother of 12 accused of keeping kids...,Thu Jun 11 16:00:29 +0000 2015,[],A pregnant mother of 12 is due in court on chi...,,"[TULSA, Okla. – A pregnant mother of 12 is du...",Pregnant mother of 12 accused of keeping kids ...
8,608229011572068352,[],[RT @fionamatthias: 10 ways the expat life Is ...,Tue Jun 09 11:07:51 +0000 2015,[Scotland to Seek Second Independence Referend...,There’s no autopilot when you're an expat livi...,"adventure,Alienation,Bangkok,Culture,Culture S...",[There’s no autopilot when you live abroad. No...,10 Ways the Expat Life Is Like a Continual Esp...
9,609046214554755072,[media/609046214554755073.jpg],"[House #GOP plans two days of debate, Friday s...",Thu Jun 11 17:15:07 +0000 2015,"[Obama, Chairman Paul Ryan and Republicans act...",House Republican leaders have planned for a tw...,,[House Republican leaders have planned for a t...,"House GOP plans two days of debate, Friday sho..."


In [24]:
#all the labels
clickbait_data1 = pd.read_json("test.json", lines=True)
clickbait_data2 = pd.read_json("truth2.json", lines=True)
clickbait_data1 = clickbait_data1.append(clickbait_data2)

In [25]:
pd_pretain=clickbait_data['postText']

In [26]:
pd_prelabel=clickbait_data1['truthClass']

In [28]:
#delete the nulll value if it exits
pd_pretain= pd_pretain.dropna()

In [29]:
#convert the text "clickbait" & "non-clickbait" to the number 0,1
labels = pd_prelabel.map(lambda x : 1 if x=='clickbait' else 0)

In [30]:
labels

0        0
1        0
2        1
3        0
4        1
5        1
6        0
7        0
8        0
9        0
10       0
11       0
12       0
13       0
14       1
15       0
16       0
17       0
18       0
19       0
20       1
21       1
22       0
23       0
24       1
25       1
26       0
27       0
28       0
29       0
        ..
19508    0
19509    0
19510    0
19511    0
19512    0
19513    1
19514    0
19515    0
19516    0
19517    0
19518    0
19519    1
19520    1
19521    0
19522    0
19523    0
19524    0
19525    1
19526    0
19527    0
19528    0
19529    0
19530    0
19531    1
19532    0
19533    0
19534    0
19535    0
19536    1
19537    0
Name: truthClass, Length: 21997, dtype: int64

In [31]:
### Text Normalizing function.
import re
from nltk import SnowballStemmer
def clean_text(text):
        
    
    ## Remove puncuation
    text = text.translate(string.punctuation)
    
    ## Convert words to lower case and split them
    text = text.lower().split()
    
    ## Remove stop words
    stops = set(stopwords.words("english"))
    text = [w for w in text if not w in stops and len(w) >= 3]
    
    text = " ".join(text)
    ## Clean the text
    text = re.sub(r"[^A-Za-z0-9^,!.\/'+-=]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ^ ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    text = re.sub(r"'", " ", text)
    text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " : ", text)
    text = re.sub(r" e g ", " eg ", text)
    text = re.sub(r" b g ", " bg ", text)
    text = re.sub(r" u s ", " american ", text)
    text = re.sub(r"\0s", "0", text)
    text = re.sub(r" 9 11 ", "911", text)
    text = re.sub(r"e - mail", "email", text)
    text = re.sub(r"j k", "jk", text)
    text = re.sub(r"\s{2,}", " ", text)

    text = text.split()
    stemmer = SnowballStemmer('english')
    stemmed_words = [stemmer.stem(word) for word in text]
    text = " ".join(stemmed_words)
    return text
new_post=[]
n=0
for text in pd_pretain:
    for i in text:
        list=[]
        i = clean_text(i)
        list.append(i)
    new_post.append(list)

In [32]:
#check if all the post were normalized
new_post

[['appl io app thin featur give phone storag boost'],
 ['kenbrown12 : emerg market investor best monti python - - run away run away'],
 ['u s soccer start answer tough question hope solo eric adelson write'],
 ['theme park like disney world left middl class behind'],
 ['could light bulb hurt health one compani put warn label bulb :'],
 ['classic 00s song actual meant artist'],
 ['dez bryant report consid skip week game doesn t get long - term deal'],
 ['pregnant mother accus keep kid wast - fill home'],
 ['fionamatthia : way expat life like continu espresso buzz via wsj'],
 ['hous gop plan two day debat friday showdown vote obama trade agenda'],
 ['azeri govern behind foreign media ban say european game offici'],
 ['one three complain unhappi nhs'],
 ['open letter jerri seinfeld polit correct colleg student'],
 ['britain forc withdraw spi russia china access file stolen snowden'],
 ['tourist detain malaysia get nake sacr mountain :'],
 ['dominiqu strauss - kahn await verdict aggrav pim

In [33]:
### Create sequence
vocabulary_size = 20000
tokenizer = Tokenizer(num_words= vocabulary_size)
tokenizer.fit_on_texts(new_post)
sequences = tokenizer.texts_to_sequences(new_post)
data = pad_sequences(sequences, maxlen=50)

print(data.shape)

(21997, 50)


In [68]:
### Model 1: Simple LSTM Model
model_lstm = Sequential()
model_lstm.add(Embedding(20000, 100, input_length=50))
model_conv.add(Dropout(0.2))
model_lstm.add(LSTM(100, dropout=0.4, recurrent_dropout=0.4))
model_lstm.add(Dense(1, activation='sigmoid'))
model_lstm.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [69]:
# Use the first small dataset with 2771 data to fit the model, and then use the another large dataset to validate the model
# The accuracy and loss were showed as follow.
model_lstm.fit(data, np.array(labels), validation_split=0.874, epochs=15)

Train on 2771 samples, validate on 19226 samples
Epoch 1/15
2771/2771 [==============================] - 12s 4ms/step - loss: 0.6209 - acc: 0.6987 - val_loss: 0.5614 - val_acc: 0.7561
Epoch 2/15
2771/2771 [==============================] - 10s 4ms/step - loss: 0.6043 - acc: 0.6994 - val_loss: 0.5725 - val_acc: 0.7561
Epoch 3/15
2771/2771 [==============================] - 10s 4ms/step - loss: 0.4972 - acc: 0.7275 - val_loss: 0.5776 - val_acc: 0.7540
Epoch 4/15
2771/2771 [==============================] - 10s 4ms/step - loss: 0.0761 - acc: 0.9892 - val_loss: 0.7172 - val_acc: 0.6207
Epoch 5/15
2771/2771 [==============================] - 11s 4ms/step - loss: 0.0227 - acc: 0.9921 - val_loss: 0.7468 - val_acc: 0.5704
Epoch 6/15
2771/2771 [==============================] - 11s 4ms/step - loss: 0.0204 - acc: 0.9906 - val_loss: 0.7514 - val_acc: 0.5790
Epoch 7/15
2771/2771 [==============================] - 11s 4ms/step - loss: 0.0195 - acc: 0.9913 - val_loss: 0.8442 - val_acc: 0.4541
Epoch 

In [66]:
### Model 2: Combine 2 CNN with LSTM
def create_conv_model():
    model_conv = Sequential()
    model_conv.add(Embedding(vocabulary_size, 100, input_length=50))
    model_conv.add(Dropout(0.2))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(Conv1D(64, 5, activation='relu'))
    model_conv.add(MaxPooling1D(pool_size=4))
    model_conv.add(Dropout(0.4))
    model_conv.add(LSTM(100))
    model_conv.add(Dense(1, activation='sigmoid'))
    model_conv.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model_conv

In [67]:
model_conv = create_conv_model()
model_conv.fit(data, np.array(labels), validation_split=0.874, epochs = 15)

Train on 2771 samples, validate on 19226 samples
Epoch 1/15
2771/2771 [==============================] - 5s 2ms/step - loss: 0.6248 - acc: 0.6987 - val_loss: 0.5604 - val_acc: 0.7561
Epoch 2/15
2771/2771 [==============================] - 3s 998us/step - loss: 0.6146 - acc: 0.6994 - val_loss: 0.5683 - val_acc: 0.7561
Epoch 3/15
2771/2771 [==============================] - 3s 936us/step - loss: 0.6140 - acc: 0.6994 - val_loss: 0.5664 - val_acc: 0.7561
Epoch 4/15
2771/2771 [==============================] - 3s 930us/step - loss: 0.6143 - acc: 0.6994 - val_loss: 0.5639 - val_acc: 0.7561
Epoch 5/15
2771/2771 [==============================] - 3s 953us/step - loss: 0.6142 - acc: 0.6994 - val_loss: 0.5637 - val_acc: 0.7561
Epoch 6/15
2771/2771 [==============================] - 3s 940us/step - loss: 0.6132 - acc: 0.6994 - val_loss: 0.5648 - val_acc: 0.7561
Epoch 7/15
2771/2771 [==============================] - 3s 953us/step - loss: 0.6122 - acc: 0.6994 - val_loss: 0.5695 - val_acc: 0.7561
E

In [42]:
df_save = pd.DataFrame(data)
df_label = pd.DataFrame(np.array(labels))

In [43]:
result = pd.concat([df_save, df_label], axis = 1)

In [44]:
result.to_csv('train_dense_word_vectors.csv', index=False)

In [45]:
embeddings_index = dict()
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


In [46]:
# create a weight matrix for words in training dataset
embedding_matrix = np.zeros((vocabulary_size, 100))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

In [48]:
### Model 3: Use the pre-trained glove 100 dimension word embeddings as initial input
### Same model structure with Model 2
model_glove = Sequential()
model_glove.add(Embedding(vocabulary_size, 100, input_length=50, weights=[embedding_matrix], trainable=False))
model_glove.add(Dropout(0.2))
model_glove.add(Conv1D(64, 5, activation='relu'))
model_glove.add(MaxPooling1D(pool_size=4))
model_glove.add(LSTM(100))
model_glove.add(Dense(1, activation='sigmoid'))
model_glove.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [50]:
model_glove.fit(data, np.array(labels), validation_split=0.874, epochs = 15)


Train on 2771 samples, validate on 19226 samples
Epoch 1/15
2771/2771 [==============================] - 3s 1ms/step - loss: 0.6129 - acc: 0.6994 - val_loss: 0.5596 - val_acc: 0.7561
Epoch 2/15
2771/2771 [==============================] - 3s 1ms/step - loss: 0.6138 - acc: 0.6994 - val_loss: 0.5582 - val_acc: 0.7561
Epoch 3/15
2771/2771 [==============================] - 3s 1ms/step - loss: 0.6128 - acc: 0.6994 - val_loss: 0.5688 - val_acc: 0.7561
Epoch 4/15
2771/2771 [==============================] - 3s 1ms/step - loss: 0.6129 - acc: 0.6994 - val_loss: 0.5653 - val_acc: 0.7561
Epoch 5/15
2771/2771 [==============================] - 3s 1ms/step - loss: 0.6127 - acc: 0.6994 - val_loss: 0.5661 - val_acc: 0.7561
Epoch 6/15
2771/2771 [==============================] - 3s 1ms/step - loss: 0.6124 - acc: 0.6994 - val_loss: 0.5585 - val_acc: 0.7561
Epoch 7/15
2771/2771 [==============================] - 3s 1ms/step - loss: 0.6135 - acc: 0.6994 - val_loss: 0.5581 - val_acc: 0.7561
Epoch 8/15
27

In [51]:
lstm_embds = model_lstm.layers[0].get_weights()[0]

In [52]:
conv_embds = model_conv.layers[0].get_weights()[0]

In [53]:
glove_emds = model_glove.layers[0].get_weights()[0]

In [54]:
word_list = []
for word, i in tokenizer.word_index.items():
    word_list.append(word)

In [55]:
def plot_words(data, start, stop, step):
    trace = go.Scatter(
        x = data[start:stop:step,0], 
        y = data[start:stop:step, 1],
        mode = 'markers',
        text= word_list[start:stop:step]
    )
    layout = dict(title= 't-SNE 1 vs t-SNE 2',
                  yaxis = dict(title='t-SNE 2'),
                  xaxis = dict(title='t-SNE 1'),
                  hovermode= 'closest')
    fig = dict(data = [trace], layout= layout)
    py.iplot(fig)

In [56]:
lstm_tsne_embds = TSNE(n_components=2).fit_transform(lstm_embds)

In [57]:
### Use TSNE to see the distribution of the word embedding result to explore the reason of different accuracy.
plot_words(lstm_tsne_embds, 0, 2000, 1)

In [58]:
conv_tsne_embds = TSNE(n_components=2).fit_transform(conv_embds)

In [59]:
plot_words(conv_tsne_embds, 0, 2000, 1)


In [60]:
glove_tsne_embds = TSNE(n_components=2).fit_transform(glove_emds)

In [61]:
plot_words(glove_tsne_embds, 0, 2000, 1)